In [1]:
# Importing libraries
import ee
import pandas as pd
import matplotlib.pyplot as plt
ee.Initialize() # Initalizing Google Earth Engine (GEE)

In [2]:
# Creating a list for Geometry of place 1 from Q Paper
place1 = [[75.45157492160797,30.634404129904425],
[75.4524278640747,30.63372099804389],
[75.45236885547638,30.634399514164798],
[75.45157492160797, 30.634404129904425]]


# Creating a list for Geometry of place 2 from Q Paper
place2 = [[85.85622847080231, 26.09425078918021],
[85.85590660572052, 26.093581136401006],
[85.85663080215454, 26.09337879451938],
[85.85686147212981, 26.094009907326967],
[85.85622847080231, 26.09425078918021]]

# Creating a list for Geometry of place 3 from Q Paper
place3 = [[78.66571158170699, 17.66869116558751],
[78.6662346124649, 17.6686911655875],
[78.6662346124649, 17.66929686130703],
[78.66571158170699, 17.66929686130703],
[78.66571158170699, 17.66869116558751]]

In [3]:
'''
    Below function is written to calculate the mean MSAVI2 for each image in the image collection and 
    creating features
'''

def MSAVI2(img):  
    nir = img.select('B8') # from Image selecting B8 band
    
    red = img.select('B4')  # from Image selecting B4 band
    
    msavi2 = img.expression(
       '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2',
        {
         'NIR': nir,
         'RED': red,
         }).rename('MSAVI2') # Calculting MSAVI2 
    
    msavi2Value = msavi2.reduceRegion(geometry= FeatureCollection,reducer = ee.Reducer.mean()).get('MSAVI2') # Performing mean for geometry area
    
    return ee.Feature(None, {'MSAVI2': msavi2Value}).copyProperties(img, ['system:time_start']) #creating a feature with MSAVI2 values and date time


In [4]:
'''
    Below function is written to convert feature collection to pandas dataframe
'''

def fc2df(FeatureCollection):
    Features = FeatureCollection.getInfo()['features'] # from feature collection extracting features
    
    dictarr = [] # Creating a list to collect properties in the feature which consist of MSAVI2 values and System time 
    
    for f in Features: # Iterating through each feature
        
        attr = f['properties'] # extracting MSAVI2 and system time from propertiesof each feature
        
        dictarr.append(attr) # appending to dictarr list
        
    df = pd.DataFrame(dictarr) # creating a dataframe from list 
    
    return df

In [5]:
start_date = '2020-01-01' ### Start Date as mentioned in Q Paper
end_date = '2020-06-30' ### End Date as mentioned in Q Paper


geoms = [place1,place2,place3] ### Created a list for the 3 places geometries mentioned in Q Paper

n = 1 
for geom in geoms: ### Iterating through each place
    FeatureCollection = ee.Geometry.Polygon(geom) ### Creating the Polygon in required format for Google Earth Engine(GEE)
    
    ########## Calling data for the given Start and End Date  ############
    collection = (ee.ImageCollection('COPERNICUS/S2') # selecting sentinel 2-L1C dataset in GEE (image collection)
                  .filterBounds(FeatureCollection) # Searching based on location
                  .filter(ee.Filter.date(start_date, end_date)) # Extracting data between Start and End Date
                  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 50)) # Applying a cloud filter lessthan 50 %


    ########## Calculating NDVI Mean for given geom ###################

    msavi2col = collection.map(MSAVI2) # mapping to MSAVI1 function to extract mean for each image in collection
    
    msavi2 = fc2df(msavi2col) # calling fc2df function, converting features to dataframe
    
    msavi2['Date'] = pd.to_datetime(pd.to_datetime(
        msavi2['system:time_start'], unit='ms').astype(str).str[:10]) # creating Date column and converting timestamp(ms) to datetime format
    
    msavi2 = msavi2.drop_duplicates(subset = ['Date']) # Dropping the redundant information
    
    msavi2.drop(columns = ['system:time_start'],inplace = True) # Dropping column with date in timestamp(ms)
    
    msavi2.rename(columns = {'MSAVI2':'place_'+str(n)},inplace = True) # renaming columns name with MSAVI2 with place name
    
    msavi2 = msavi2.dropna() # dropping null rows
    
    ############################ Plotting using matplotlib ############################
    plt.ioff() ## Turning off the interactive mode
    fig,ax = plt.subplots(figsize = (28,15)) # creating a fig and axis 
    ax.plot(msavi2.Date.values,msavi2[msavi2.columns[0]].values) ## plotting the MSAVI2 time series alond date x-axis 
    ax.set_title('MSAVI2 Time Series Analysis for Place '+str(n),fontsize=30) # Adding title to plot
    plt.xlabel("Date",fontsize=20) 
    plt.ylabel("Index Values",fontsize=20)
    plt.legend(['MSAVI2']) # adding legend to plot
    plt.savefig('../images/place_'+str(n)+'.png') # saving figure to loaction in png format
    plt.close() # closing the session to remove from memory
    n = n+1
